In [7]:
import os
import numpy as np
import pandas as pd
import pickle
import numpy as np
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from sklearn import linear_model
from lightgbm import LGBMRegressor 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

In [8]:
infect_A=pd.read_csv('infectDataset/city_A/infection.csv',header=None)
infect_A.columns = ['city','region_id','date', 'index']
infect_B=pd.read_csv('infectDataset/city_B/infection.csv',header=None)
infect_B.columns = ['city','region_id','date', 'index']
infect_C=pd.read_csv('infectDataset/city_C/infection.csv',header=None)
infect_C.columns = ['city','region_id','date', 'index']
infect_D=pd.read_csv('infectDataset/city_D/infection.csv',header=None)
infect_D.columns = ['city','region_id','date', 'index']
infect_E=pd.read_csv('infectDataset/city_E/infection.csv',header=None)
infect_E.columns = ['city','region_id','date', 'index']
infect_F=pd.read_csv('infectDataset/city_F/infection.csv',header=None)
infect_F.columns = ['city','region_id','date', 'index']
infect_G=pd.read_csv('infectDataset/city_G/infection.csv',header=None)
infect_G.columns = ['city','region_id','date', 'index']
infect_H=pd.read_csv('infectDataset/city_H/infection.csv',header=None)
infect_H.columns = ['city','region_id','date', 'index']
infect_I=pd.read_csv('infectDataset/city_I/infection.csv',header=None)
infect_I.columns = ['city','region_id','date', 'index']
infect_J=pd.read_csv('infectDataset/city_J/infection.csv',header=None)
infect_J.columns = ['city','region_id','date', 'index']
infect_K=pd.read_csv('infectDataset/city_K/infection.csv',header=None)
infect_K.columns = ['city','region_id','date', 'index']
#
infection_data={'A':infect_A,'B':infect_B,'C':infect_C,'D':infect_D,'E':infect_E,'F':infect_F,'G':infect_G,'H':infect_H,'I':infect_I,'J':infect_J,'K':infect_K}
#按照date进行groupby,total_A就表示了city_A 45天每天的新增人数
grouped=infect_A.groupby(['date'])
total_A=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_B.groupby(['date'])
total_B=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_C.groupby(['date'])
total_C=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_D.groupby(['date'])
total_D=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_E.groupby(['date'])
total_E=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_F.groupby(['date'])
total_F=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_G.groupby(['date'])
total_G=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_H.groupby(['date'])
total_H=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_I.groupby(['date'])
total_I=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_J.groupby(['date'])
total_J=grouped['index'].agg(np.sum).reset_index(drop=True)
grouped=infect_K.groupby(['date'])
total_K=grouped['index'].agg(np.sum).reset_index(drop=True)
city_infect={'A':total_A,'B':total_B,'C':total_C,'D':total_D,'E':total_E,'F':total_F,'G':total_G,'H':total_H,'I':total_I,'J':total_J,'K':total_K}

In [ ]:
#这段代码
   
model_lgbm = LGBMRegressor(n_estimators=500, metric='mae', random_state=1234, min_child_samples=5, min_child_weight=0.000001)
re_sub=pd.read_csv('submits/submissionShilft1_16.csv',header=None)
re_sub.columns = ['city','region_id','date', 'index']
pre_city_id={}
pre_id={}
def find_zero(data):
    for i in range(len(data)):
        if data[i]>0:
            return i
def calculate_lag(df, lag_list, column):
    for lag in lag_list:
        column_lag = column + "_" + str(lag)
        df[column_lag] = df[column].shift(lag, fill_value=0)
    return df

def calculate_trend(df, lag_list, column):
    for lag in lag_list:
        trend_column_lag = "Trend_" + column + "_" + str(lag)
        df[trend_column_lag] = (df[column].shift(0, fill_value=0) - 
                                df[column].shift(lag, fill_value=0))/df[column].shift(lag, fill_value=0.001)
    return df

def get_feature(df):
    df = calculate_lag(df, range(1,7), 'index')
    #all_data = calculate_lag(all_data, range(1,7), 'migration')
    df = calculate_trend(df, range(1,7), 'index')
    #all_data = calculate_trend(all_data, range(1,7), 'migration')
    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.fillna(0, inplace=True)
    return df
def get_true(city,Id):
    infection=infection_data[city]
    id_infect=infection[infection['region_id'].isin([Id])].reset_index(drop=True)
    return id_infect['index']
grouped=re_sub.groupby(['city','region_id'])
pre_dicSeir={'A':{},'B':{},'C':{},'D':{},'E':{},'F':{},'G':{},'H':{},'I':{},'J':{},'K':{}}
for name,group in grouped:
    pre_dic={}
    data=group.reset_index(drop=True)
    city=name[0]
    Id=name[1]
    true_data=get_true(city,Id)
    cut=find_zero(true_data.values)
    true_data=true_data[cut:].reset_index(drop=True)
    y=data['index']
    group=pd.concat((true_data,y),axis=0).reset_index()
    
    train_df=get_feature(group)
    X_train=train_df.drop(['index'],axis=1)
    Y_train=train_df['index']
    regr = model_lgbm#
    #regr= linear_model.LinearRegression()
    regr.fit(X_train, Y_train)
    y_pred = regr.predict(X_train)
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(true_data, c='g', lw=3, label='True')
    ax.plot(y_pred, c='b', lw=2, label='lgbFit')
    x=[i+len(true_data) for i in range(30)]
    ax.plot(x,y, c='r', lw=2, label='seir')
    ax.set_xlabel('Day',fontsize=20)
    ax.set_ylabel('Infect count', fontsize=20)
    ax.grid(1)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend();
    save_dir='SEIR_fit_city_lgbPersudo/'+city
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    plt.savefig(os.path.join(save_dir,str(Id)+'.png'))
    plt.show()
    plt.close()
    pre_dicSeir[city][Id]=y_pred[-30:]
    

#for num,city,region_id,date,index in re_sub.itertuples():
    #pre_id[region_id].append(index)
    #re_sub1.loc[num,'index']=re_sub0.loc[num,'index']

In [10]:
#生成提交结果
import math
import random
submit=pd.read_csv('submission.csv',header=None)
submit.columns = ['city','region_id','date', 'index']
#
submit=submit.drop(['index'],axis=1)
pres=[]
dic_pre=pre_dicSeir
def convert_date(date):
    if date==21200630:
        return 0
    else:
        return date-21200700
def get_result(city,region_id,date):
    if city=='A':
        pre=math.ceil(dic_pre['A'][region_id][convert_date(date)])
    if city=='B':
        pre=math.ceil(dic_pre['B'][region_id][convert_date(date)])
    if city=='C':
        pre=math.ceil(dic_pre['C'][region_id][convert_date(date)])
    if city=='D':
        pre=math.ceil(dic_pre['D'][region_id][convert_date(date)])
    if city=='E':
        pre=math.ceil(dic_pre['E'][region_id][convert_date(date)])
    if city=='F':
        pre=math.ceil(dic_pre['F'][region_id][convert_date(date)])
    if city=='G':
        pre=math.ceil(dic_pre['G'][region_id][convert_date(date)])
    if city=='H':
        pre=math.ceil(dic_pre['H'][region_id][convert_date(date)])
    if city=='I':
        pre=math.ceil(dic_pre['I'][region_id][convert_date(date)])
    if city=='J':
        pre=math.ceil(dic_pre['J'][region_id][convert_date(date)])
    if city=='K':
        pre=math.ceil(dic_pre['K'][region_id][convert_date(date)])
    return pre
#
for _,city,region_id,date in submit.itertuples():
    pre=get_result(city,region_id,date)
    if pre<0:
        pre=0
    pres.append(int(pre))

submit['index']=pres
submit_dir='submits/'
if not os.path.exists(submit_dir):
    os.makedirs(submit_dir)
submit.to_csv(os.path.join(submit_dir,'subLgbPersudo.csv'),index=False,header=False)
submit.head(2)

,city,region_id,date,index
0,A,0,21200630,3
1,A,0,21200701,3
